In [ ]:
# 使用嵌入法进行logistic 的特征工程
from sklearn.linear_model import LogisticRegression as LR
from sklearn.datasets import load_breast_cancer
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
data = load_breast_cancer()
data.data.shape
LR_ = LR(solver='liblinear', C=0.9, random_state=420)
cross_val_score(LR_, data.data, data.target, cv=10).mean()
# 使用L1范式进行筛选，删掉在范式下判定无效的特征
x_embedded = SelectFromModel(LR_, norm_order=1).fit_transform(data.data, data.target)
x_embedded.shape

# 改良，对threshold加上改变，看看每个系数这个重要性的变化
fullx = []
fsx = []
# 这时我们选择的指标就是逻辑回归中的系数而不是L1范数
threshold = np.linspace(0, abs(LR_.fit(data.data, data.target).coef_).max(), 20)
k = 0
for i in threshold:
    x_embedded = SelectFromModel(LR_, threshold=i).fit_transform(data.data, data.target)
    fullx.append(cross_val_score(LR_, data.data, data.target, cv=4).mean())
    fsx.append(cross_val_score(LR_, x_embedded, data.target, cv=5).mean())
    print((threshold[k], x_embedded.shape[1]))
    k += 1
plt.figure(figsize=(20, 5))
plt.plot(threshold, fullx, label='full')
plt.plot(threshold, fsx, label='feature selection')
plt.xticks(threshold)
plt.legend()
plt.show()    
###调整二， 调整C
C = np.arange(0.01, 10.01, 0.5)#C = .0.666

In [ ]:
LR_ = LR(solver="liblinear", C=6.06, random_state=420)
cross_val_score(LR_, data.data, data.target, cv=10).mean()#0.97
LR_ = LR(solver="liblinear", C=6.06, random_state=420)
x_embedded = SelectFromModel(LR_, norm_order=1).fit_transform(data.data, data.target)
cross_val_score(LR_, x_embedded, data.target, cv=10).mean()
x_embedded.shape#特征只剩下9个

In [ ]:
# 乳腺癌数据分析
from sklearn.datasets import load_breast_cancer
data = load_breast_cancer()
x = data.data
y = data.target
l2 = []
l2test = []

x_train, x_valid, y_train, y_valid = train_test_split(x, y, test_size=0.3, random_state=420)
for i in np.arange(1, 201, 10):
    # 迭代次数
    lrl2 = LR(penalty="l2", solver="liblinear", C=0.8, max_iter=i)
    lrl2 = lrl2.fit(x_train, y_train)
    l2.append(accuracy_score(lrl2.predict(x_train), y_train))
    l2test.append(accuracy_score(lrl2.predict(x_valid), y_valid))

graph = [l2, l2test]
color = ["black", "gray"]
label = ["L2", "L2test"]
plt.figure(figsize=(20, 5))
for i in range(len(graph)):
    plt.plot(np.arange(1, 201, 10), graph[i], color=color[i], label=label[i])
plt.legend(loc=4)
plt.xticks(np.arange(1, 201, 10))
plt.show()

In [ ]:
# 我们可以使用属性.n_iter_来调用本次求解中的真正需要的迭代次数
lr = LR(penalty="l2", solver="liblinear", C=0.9, max_iter=300).fit(x_train, y_train)
lr.n_iter_
# 当参数是libinearzhong ,可能斯塔未收敛 sag,但是只要模型的训练和预测效果不错，就不用再增大max——iter的数目，无所谓说是没收敛，万一收敛了还可能过度收敛



In [ ]:
# 比较分类器
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
iris = load_iris()
for multi_class in ("multinomial", "ovr"):
    clf = LogisticRegression(solver="sag", max_iter=100, random_state=42, multi_class=multi_class).fit(iris.data, iris.target)
# 打印两种multi_class模式下的训练分数
# %,代替字符串，%.3f代表保留三位小数，%s是字符串的意思
# % 后面用元组来存储变量()
    print("training score: %.3f (%s)" % (clf.score(iris.data, iris.target), multi_class))


